##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
%pip install pandas 
%pip install matplotlib
%pip install scikit-learn
%pip install scikit-optimize
%pip install seaborn
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .csv file inside that. A relative path *must* be used when loading data into pandas

In [1]:
# Can have as many cells as you want for code
import pandas as pd
filepath = "./data/catA_train.csv" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [2]:
###...code...###
import pandas as pd
import numpy as np
import geopy as geo

In [ ]:
df2 = pd.read_csv('./data/catA_train.csv')

# Descriptive Statistics

In [ ]:
# view first 5 rows of data
df2.head()

In [ ]:
df2.describe()

In [ ]:
# data types of each column
df2.dtypes

In [ ]:
df2.info()

In [ ]:
# shape of data
df2.shape

In [ ]:
df2.isna().sum()

In [ ]:
# check for duplicates
df2.duplicated().sum()

# Data Cleaning

In [ ]:
# filter out rows where latitude or longitude are null values

df2[df2['LATITUDE'].isnull() | df2['LONGITUDE'].isnull()]

In [ ]:
# drop Square Footage as values are all NULL
df2.drop(columns = ["Square Footage"])

In [ ]:
# removes rows without either lat or long coordinates, since they form a small percentage of our dataset
df2 = df2.dropna(subset=["LATITUDE", "LONGITUDE"])

In [ ]:
df2.isna().sum()

In [ ]:

# interpolate the NaN values for the columns using KNNImputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)

df2[['Year Found', 'Employees (Single Site)', 'Employees (Domestic Ultimate Total)', 'Employees (Global Ultimate Total)']]\
= imputer.fit_transform(df2[['Year Found', 'Employees (Single Site)', 'Employees (Domestic Ultimate Total)', 'Employees (Global Ultimate Total)']])

In [ ]:
df2.isna().sum()

In [ ]:
#removing inactive companies
df2 = df2[df2['Company Status (Active/Inactive)'] == 'Active']

#format SIC code as strings
df2["SIC Code"] = df2["SIC Code"].astype(str)

In [ ]:
#plot correlation matrix between variables

numerical_df = df2.select_dtypes(include=['float64', 'int64'])

# Create a correlation matrix
correlation_matrix = numerical_df.corr()

# Assuming correlation_matrix is your correlation matrix
correlation_df = pd.DataFrame(correlation_matrix)

# Print or use the correlation DataFrame as needed
correlation_df

In [ ]:
#checking for consistency between SIC Code and Industry
sic_to_industry_mapping = {}
for index, row in df2.iterrows():
    sic_code = row['SIC Code']
    industry = row['Industry']
    if sic_code not in sic_to_industry_mapping:
        sic_to_industry_mapping[sic_code] = industry
    else:
        # If the SIC code is already in the mapping, check if the mapped industry is the same
        if sic_to_industry_mapping[sic_code] != industry:
            print(f"Warning: SIC code {sic_code} maps to multiple industries: {sic_to_industry_mapping[sic_code]} and {industry}")

# Check for duplicates in the mapped industries
mapped_industries = list(sic_to_industry_mapping.values())
duplicated_industries = [industry for industry in mapped_industries if mapped_industries.count(industry) > 1]

if len(duplicated_industries) == 0:
    print("None of the mapped industries appear more than once.")
else:
    print("The following mapped industries appear more than once:")
    print(duplicated_industries)

Photographic Equipment and Supplies have two codes of 3861 and 5043

Opthalmic Goods have two codes of 3851 and 5048

In [ ]:
# Create a mapping between 8-Digit SIC Codes and descriptions (industries)
sic_to_description_mapping = {}
for index, row in df2.iterrows():
    sic_code = row['8-Digit SIC Code']
    description = row['8-Digit SIC Description']
    if sic_code not in sic_to_description_mapping:
        sic_to_description_mapping[sic_code] = description
    else:
        # If the SIC code is already in the mapping, check if the mapped description is the same
        if sic_to_description_mapping[sic_code] != description:
            print(f"Warning: SIC code {sic_code} maps to multiple descriptions: {sic_to_description_mapping[sic_code]} and {description}")

# Check for duplicates in the mapped descriptions
mapped_descriptions = list(sic_to_description_mapping.values())
duplicated_descriptions = [description for description in mapped_descriptions if mapped_descriptions.count(description) > 1]

if len(duplicated_descriptions) == 0:
    print("None of the mapped descriptions appear more than once.")
else:
    print("The following mapped descriptions appear more than once:")
    print(duplicated_descriptions)

# Data Visualization

In [ ]:
#industry by domestic sales
import matplotlib.pyplot as plt
# Group the data by 'Industry' and calculate total sales for each industry
industry_sales = df2.groupby('Industry')['Sales (Domestic Ultimate Total USD)'].mean()

# Sort industries by total sales in descending order
richest_industries = industry_sales.sort_values(ascending=False)

# Plot the richest industries using a bar chart
plt.figure(figsize=(10, 6))
richest_industries.head(10).plot(kind='bar', color='skyblue')
plt.title(label = 'Top 10 Richest Industries by Average Domestic Total Sales')
plt.xlabel('Industry')
plt.ylabel('Total Sales (USD)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Above is a bar chart of the top 10 richest industries by Average Domestic Sales. Petroleum Bulk Stations And Terminals have the highest average domestic sales by an extremely large margin, followed by cordage and twine and then petroleum refining. This bar chart serves to show which industries have the highest sales in Singapore, and which industry is the most profitable in Singapore.

In [ ]:
#industry by global sales
import matplotlib.pyplot as plt
# Group the data by 'Industry' and calculate total sales for each industry
industry_sales = df2.groupby('Industry')['Sales (Global Ultimate Total USD)'].mean()

# Sort industries by total sales in descending order
richest_industries = industry_sales.sort_values(ascending=False)

# Plot the richest industries using a bar chart
plt.figure(figsize=(10, 6))
richest_industries.head(10).plot(kind='bar', color='skyblue')
plt.title('Top 10 Richest Industries by Average Global Total Sales')
plt.xlabel('Industry')
plt.ylabel('Total Sales (USD)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Above is a bar chart of the top 10 Richest Industries by Average Global Total Sales. Petroleum Refining is the highest, followed by Plastic Bottles and then Air Courier Services. The gap between Petroleum Refining and Plastic Bottles is no where near as large as the gap between Petroleum Bulk Stations And Terminals for average domestic sales. This bar chart serves to show which industries have the highest sales in the world, and which industry is the most profitable in Singapore.

In [ ]:
# Domestic sales by entity type

domestic_by_entity = df2.groupby('Entity Type')['Sales (Domestic Ultimate Total USD)'].mean()

# sort highest earning entities
richest_entity = domestic_by_entity.sort_values(ascending = False)

# plot bar graph
plt.figure(figsize = (10,8))
bp = richest_entity.plot(kind = "bar", color = "skyblue", )

plt.xlabel('Entity Type')
plt.ylabel('Total Sales (USD)')
plt.title("Average Sales (Domestic Ultimate Total USD) by Entity Type")
plt.xticks(rotation = 0)
for bar in bp.patches:
  bp.annotate(format(bar.get_height(), '.2f'),
              (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')
plt.show()

Above is the bar graph of average sales by domestic ultimate total USD by entity type. This is to identify which industry has the highest domestic sale. Subsidiary has the highest, followed closely by Branch, and then a sharp decrease in sales from Branch to Independent, and then followed closely by Parent.

In [ ]:
# Global sales by entity type

global_by_entity = df2.groupby('Entity Type')['Sales (Global Ultimate Total USD)'].mean()

# sort highest earning entities
richest_global_entity = global_by_entity.sort_values(ascending = False)

# plot bar graph
plt.figure(figsize = (10,8))
bp2 = richest_global_entity.plot(kind = "bar", color = "skyblue", )

plt.xlabel('Entity Type')
plt.ylabel('Total Sales (USD)')
plt.title("Average Sales (Global Ultimate Total USD) by Entity Type")
plt.xticks(rotation = 0)
for bar in bp2.patches:
  bp2.annotate(format(bar.get_height(), '.2f'),
              (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')
plt.show()

Above is the bar graph of average sales by global ultimate total USD by entity type. This is to identify which industry has the highest sales internationally. Independent has the highest, followed closely by Subsidiary, and then a sharp decrease in sales from Branch to Independent, and then followed closely by Parent.

In [ ]:
# map latitude and longitude on a scatterplot, with opacity indicating Sales (Domestic Ultimate Total USD)
from matplotlib.colors import LogNorm
import seaborn as sns
norm = LogNorm()

# create new variable Region
conditions = [((df2["LATITUDE"]) > 1.3667) & ((df2["LONGITUDE"]) > 103.8),
              ((df2["LATITUDE"]) > 1.3667) & ((df2["LONGITUDE"]) <= 103.8),
              ((df2["LATITUDE"]) <= 1.3667) & ((df2["LONGITUDE"]) > 103.8),
              ((df2["LATITUDE"]) <= 1.3667) & ((df2["LONGITUDE"]) <= 103.8)]

values = ["North-East", "South-East", "North-West", "South-West"]

df2['Region'] = np.select(conditions, values)
print(df2["Region"].value_counts())
#Find top 10 highest earning industries in terms
total_industry_sales = df2.groupby('Industry')['Sales (Global Ultimate Total USD)'].sum()
top10_richest_industries = total_industry_sales.sort_values(ascending=False).head(10)

#Assigning colours to the top 10 industries
ten_colours = sns.color_palette("Set2", n_colors = 10)
colourmap = dict(zip(top10_richest_industries.index, ten_colours))

#filter for rows with industries in the top 10
location_data = df2[df2["Industry"].isin(top10_richest_industries.index.to_list())]

#plot a scatterplot of longitude and latitude, with the opacity indicating global sales amount, colour indicating industry
plt.figure(figsize = (10,8))
plt.scatter(location_data["LATITUDE"], location_data["LONGITUDE"], \
            alpha = norm(location_data["Sales (Global Ultimate Total USD)"]),\
            c = location_data["Industry"].map(colourmap))
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.title("Map and Sales (Global Ultimate Total USD) of Companies in Top 10 Industries by Sales (Global Ultimate Total USD)")

#legend

legend_labels = location_data["Industry"].unique()
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=industry,
                              markerfacecolor=colourmap[industry], markersize=4) for industry in legend_labels]
plt.legend(handles = legend_handles, title='Industry', prop={'size': 5}, loc='upper left', bbox_to_anchor=(1, 1))

Above is a distribution of companies in the top 10 industries by sum of global sales on the map of Singapore. The colour identifies each industry and the opacity indicates the amount of sales per company. Most of the companies in Singapore are under offices of holding companies not elsewhere classified. Most of the higher earning companies are in the NorthWest region of Singapore.

In [ ]:
df2.columns

In [ ]:
plt.boxplot(
    x=df2['Employees (Domestic Ultimate Total)']
)

Most of the companies employ between 10k to 20k employees, with with only a few companies having enough resources to hire between 20k to 80k employees.

In [ ]:
df_copy = df2[df2['Employees (Domestic Ultimate Total)'] > 20000]

In [ ]:
import seaborn as sns

sns.histplot(
    data=df_copy,
    x='Employees (Domestic Ultimate Total)',
    fill=True,
    kde=True,
    bins=5,
    stat='percent'
)

plt.title('Histogram of Employees (Domestic Ultimate Total)')
plt.show()

More than 50% of companies hire lesser than 35k, with only 10% of companies hiring more than 65k.

In [ ]:
df2.boxplot(
    column='Employees (Global Ultimate Total)'
)

This boxplot shows that majority of companies higher lesser than 1 million employees.

In [ ]:
sns.kdeplot(
    data=df2,
    x='Employees (Global Ultimate Total)',
    log_scale=True
)

plt.title('KDE plot of Employees (Global Ultimate Total)')
plt.show()

df2.boxplot(
    column='Sales (Domestic Ultimate Total USD)'
)

In [ ]:
sns.kdeplot(
    data=df2,
    x='Sales (Domestic Ultimate Total USD)',
    log_scale=True
)

plt.title('KDE plot of Sales (Domestic Ultimate Total USD)')
plt.show()

In [ ]:
df2.boxplot(
    column='Employees (Single Site)'
)

In [ ]:
df2.boxplot(
    column='Employees (Domestic Ultimate Total)'
)

In [ ]:
df2.boxplot(
    column='Is Domestic Ultimate'
)

In [ ]:
df2.boxplot(
    column='Is Global Ultimate'
)

In [ ]:
domestic_total_by_industry = df2.groupby('Industry')['Sales (Domestic Ultimate Total USD)'].sum()
#print(domestic_total_by_industry)

# Filter DataFrame for companies considered as domestic ultimate
domestic_ultimate_df = df2[df2['Is Domestic Ultimate'] == 1]
domestic_ultimate_total_by_industry = domestic_ultimate_df.groupby('Industry')['Sales (Domestic Ultimate Total USD)'].sum()


new_table = pd.concat([domestic_total_by_industry, domestic_ultimate_total_by_industry[1:]], axis=1)
new_table.columns = ['Total Sales', 'Domestic Ultimate Total Sales']
#print(new_table.head(5))
# Calculate proportion
new_table["proportion contributed by domestic ultimate"] = new_table["Domestic Ultimate Total Sales"]/new_table["Total Sales"]
print(new_table)

In [ ]:
global_total_by_industry = df2.groupby('Industry')['Sales (Global Ultimate Total USD)'].sum()
#print(domestic_total_by_industry)

# Filter DataFrame for companies considered as domestic ultimate
global_ultimate_df = df2[df2['Is Global Ultimate'] == 1]
global_ultimate_total_by_industry = global_ultimate_df.groupby('Industry')['Sales (Global Ultimate Total USD)'].sum()


new_table = pd.concat([global_total_by_industry, global_ultimate_total_by_industry[1:]], axis=1)
new_table.columns = ['Total Sales', 'Global Ultimate Total Sales']
#print(new_table.head(5))
# Calculate proportion
new_table["proportion contributed by Global ultimate"] = new_table["Global Ultimate Total Sales"]/new_table["Total Sales"]
print(new_table)

# Feature Selection

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
columns_to_drop = ['LATITUDE', 'LONGITUDE', 'AccountID', 'Company', '8-Digit SIC Code', 'SIC Code', '8-Digit SIC Description', 'Parent Company', 'Employees (Single Site)', 'Employees (Domestic Ultimate Total)', 'Employees (Global Ultimate Total)', 'Square Footage', 'Import/Export Status', 'Fiscal Year End', 'Company Description', 'Domestic Ultimate Company']

df2.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df2.dropna(inplace=True)

In [ ]:
# scale numeric features
numeric_cols = ['Year Found', 'Sales (Domestic Ultimate Total USD)']

scaler = StandardScaler()
df2[numeric_cols] = scaler.fit_transform(df2[['Year Found', 'Sales (Domestic Ultimate Total USD)']])

In [ ]:
label_encoder = LabelEncoder()

cols_to_label_encode = ['Industry', 'Parent Country', 'Global Ultimate Company', 'Global Ultimate Country', 'Entity Type', 'Ownership Type', 'Company Status (Active/Inactive)', 'Region']

# label encode the columns to label encode
df2[cols_to_label_encode] = df2[cols_to_label_encode].apply(LabelEncoder().fit_transform)

In [ ]:
df2.to_csv('./data/cleaned_data.csv')

# Model Selection

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
import pandas as pd

df2 = pd.read_csv('./data/cleaned_data.csv')

X_train, X_test, y_train, y_test = train_test_split(
    df2.drop(columns='Sales (Global Ultimate Total USD)'),
    df2['Sales (Global Ultimate Total USD)'],
    test_size=0.10,
    random_state=42
  )

model = GradientBoostingRegressor()

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

In [ ]:
model.score(X_test, y_test)

In [ ]:
from skopt import BayesSearchCV

search_space = {
    'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate': [0.000001, 0.0001, 0.1, 1, 10],
    'n_estimators': [50, 100, 125, 150, 175, 200, 225, 250, 275, 300],
    'criterion': ['friedman_mse', 'squared_error'],
    'min_samples_split': [10, 50, 100, 125, 150, 200, 250, 300, 350, 400],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 40, 50, 60, 70]
}

opt = BayesSearchCV(
    GradientBoostingRegressor(),
    search_space,
    n_iter=5,
    verbose=3,
    cv=3
)

opt.fit(X_train, y_train)

In [ ]:
opt.best_params_

In [ ]:
model_final = GradientBoostingRegressor(
    criterion='squared_error',
    learning_rate=0.1,
    loss='squared_error',
    max_depth=8,
    min_samples_split=300,
    n_estimators=250
)

model_final.fit(X_train, y_train)

y_pred = model_final.predict(X_test)
mean_squared_error(y_test, y_pred)

In [ ]:
import joblib

# Save the base model to an HDF5 file
joblib.dump(model_final, 'final_model.h5')

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [11]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''

    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_csv(filepath)
test_df = test_df.drop(columns=['Sales (Domestic Ultimate Total USD)'])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!